# 주유소 10곳이상 기름값 지도위에 표시하기

**오피넷 - 경로별**   
- 출발지: 서울, 연신내역
- 도착지: 서울, 호서직업전문학교 - 주유소 3개
- 도착지: 서울, 서울역 - 주유소 1개
- 도착지: 서울, 김포국제공항 - 주유소 4개
- 도착지: 서울, 여의도역 - 주유소 5개

In [1]:
import sys
import time
import numpy as np
import pandas as pd

# Unix style pathname pattern expansion
from glob import glob

# A browser automation framework and ecosystem.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller 

# Beautiful Soup is a library that makes it easy to scrape information from web pages.
from bs4 import BeautifulSoup

# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm
from tqdm import tqdm_notebook

## Crawling

In [2]:
path = chromedriver_autoinstaller.install() 
driver = webdriver.Chrome(path)
driver.switch_to.window(driver.window_handles[0])

# 메인페이지 접속
driver.get("http://www.opinet.co.kr/user/main/mainView.do")

# 싼주유소찾기 이동
wait = WebDriverWait(driver, 10)
quick_ul = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#quick_ul')))
quick_ul.find_element_by_css_selector('li:nth-child(2) a').click()

# 경로별 페이지 이동
wait = WebDriverWait(driver, 10)
header_path_gsi = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#container .header_path_gsi')))
header_path_gsi.find_element_by_css_selector('li.mid2 a').click()

# 출발지, 도착지 설정
destinations = ['호서직업전문학교', '서울역', '김포국제공항', '여의도역']
departures = ['연신내'] * len(destinations)

for index, destination in tqdm(enumerate(destinations)):

    print('')
    print(destination)

    # 경로별 페이지 설정
    wait = WebDriverWait(driver, 10)
    form = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searrouteVO')))

    # 경로별 설정 - 출발지
    departure = departures[index]

    element = form.find_element_by_id('search1_2')
    driver.execute_script("arguments[0].style.display = 'none';", element)

    time.sleep(1)
    Select(form.find_element_by_id('SIDO_CD1')).select_by_visible_text('서울')
    form.find_element_by_id('poiName1').send_keys('')
    form.find_element_by_id('poiName1').send_keys(departure)
    form.find_element_by_id('poiName1').send_keys(Keys.ENTER)

    # 경로별 설정 - 도착지
    element = form.find_element_by_id('search2_2')
    driver.execute_script("arguments[0].style.display = 'none';", element)

    time.sleep(1)
    Select(form.find_element_by_id('SIDO_CD2')).select_by_visible_text('서울')
    form.find_element_by_id('poiName2').send_keys('')
    form.find_element_by_id('poiName2').send_keys(destination)
    form.find_element_by_id('poiName2').send_keys(Keys.ENTER)

    # 경로별 설정 - 조회
    time.sleep(3)
    driver.find_element_by_id('searrouteSelect').click()

    # 경로별 설정 - 데이터 
    time.sleep(3)
    wait = WebDriverWait(driver, 10)
    table = wait.until(EC.presence_of_element_located((By.ID, 'osList')))
    raw = table.find_elements_by_css_selector('tr')
    
    data = []

    for row in raw:

        time.sleep(1)
        row.find_element_by_css_selector('a').click()
        
        time.sleep(3)
        os_dtail_info = driver.find_element_by_id('os_dtail_info')

        tmp = {
            '이름': { 'id':'os_nm', 'type':'string', 'value':'' },
            '전화번호': { 'id':'phn_no', 'type':'string', 'value':'' },
            '주소': { 'id':'rd_addr', 'type':'string', 'value':'' },
            '상표': { 'id':'poll_div_nm', 'type':'string', 'value':'' },
            '고급휘발유': { 'id':'bo34_p', 'type':'integer', 'value':'' },
            '보통휘발유': { 'id':'b027_p', 'type':'integer', 'value':'' },
            '경유': { 'id':'d047_p', 'type':'integer', 'value':'' },
            '실내등유': { 'id':'c004_p', 'type':'integer', 'value':'' },
            '자동차부탄': { 'id':'k015_p', 'type':'integer', 'value':'' },
        }

        time.sleep(1)
        for k, values in tmp.items():
            try:
                tmp[k].update({ 'value': os_dtail_info.find_element_by_id(values['id']).text })
            except Exception as e:
                if tmp[k]['type'] == 'integer':
                    tmp[k].update({ 'value': np.nan })
                print(k, e, sep=', ', end='\n')
        
        tmp2data = {}
        for k, values in tmp.items():
            tmp2data.update({ k: values['value'] })

        data.append(tmp2data)

    # 경로별 설정 - 저장
    df = pd.DataFrame(data)
    df.to_csv('./data/fuel-{}.csv'.format(index), sep=',', encoding='utf8', index=False)

    # to refresh the browser
    time.sleep(3)
    driver.refresh()

driver.close() 
driver.quit()

0it [00:00, ?it/s]
호서직업전문학교
고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

1it [00:28, 28.08s/it]
서울역
고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

2it [00:45, 24.94s/it]
김포국제공항
고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

고급휘발유, Me

## Merging DataFrame

In [3]:
pd.read_csv('./data/fuel-0.csv')

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN
1,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN
2,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN


In [4]:
tmp_raw = []

for filepath in glob('./data/*.csv'):
    tmp = pd.read_csv(filepath)
    tmp_raw.append(tmp)

df = pd.concat(tmp_raw)
df.index = np.arange(0, len(df))
df.to_csv('./data/opinet.csv', sep=',', encoding='utf8', index=False)
df

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄,lon,lat
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN,NaN,NaN
1,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN,NaN,NaN
2,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN,NaN,NaN
3,(주)창원이엔지 홍제지점,02-379-0753,서울 서대문구 통일로 455 (홍제동),현대오일뱅크,NaN,"1,299","1,099",NaN,NaN,NaN,NaN
4,NaN,02-2665-5189,서울 강서구 개화동로 420 (개화동),현대오일뱅크,NaN,"1,264","1,064",NaN,NaN,NaN,NaN
5,(주)유카스 에너지 올림픽점,031-985-5185,경기 김포시 고촌읍 벌말로 768-70,SK에너지,NaN,"1,265","1,065",NaN,NaN,NaN,NaN
6,산성주유소,031-970-5151,경기 고양시 덕양구 행주로 59 (행주내동),현대오일뱅크,NaN,"1,275","1,075",NaN,NaN,NaN,NaN
7,(주)한진 김포공항주유소,02-2662-0422,서울 강서구 하늘길 259,S-OIL,NaN,"1,319","1,128",NaN,NaN,NaN,NaN
8,(주)가재울뉴타운주유소,070-8879-7864,서울 서대문구 모래내로 205,현대오일뱅크,NaN,"1,332","1,132",NaN,NaN,NaN,NaN
9,남정주유소,02-305-9828,서울 서대문구 모래내로 243 (남가좌동),GS칼텍스,NaN,"1,337","1,137",NaN,NaN,NaN,NaN


## Kakaomap API

In [5]:
import pandas as pd
import numpy as np
import time

# A simple, yet elegant HTTP library.
import requests
from urllib.parse import urlparse, quote

# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm

In [6]:
with open('./config/kakaomaprestapikey.txt', 'r') as f:
    API_KEY = f.read().strip()

In [7]:
df = pd.read_csv('./data/opinet.csv')
df.head(1)

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄,lon,lat
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN,NaN,NaN


In [8]:
def getCoords(query = ''):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = { 
        'query': query
    }
    headers = { "Authorization": "KakaoAK " + API_KEY }
    api = requests.get(url, params=params, headers=headers).json()
    coords = {}
    coords.update({ 'x': float(api['documents'][0]['x']) })
    coords.update({ 'y': float(api['documents'][0]['y']) })
    return coords

In [9]:
df['lon'] = np.nan # 경도
df['lat'] = np.nan # 위도
for i in tqdm(df.index):
    try:
        addr = df['주소'][i]
        coords = getCoords(addr)
        df['lon'][i] = coords['x']
        df['lat'][i] = coords['y']
    except Exception as e:
        print(i, e)

100%|██████████| 33/33 [00:08<00:00,  3.92it/s]


In [10]:
df.dropna(subset=['lon', 'lat'], inplace=True)
df.index = np.arange(0, len(df))
df.to_csv('./data/kakaomapapi.csv', sep=',', encoding='utf8', index=False)
df.head()

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄,lon,lat
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN,126.846965,37.549279
1,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN,126.914397,37.595614
2,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN,126.913936,37.594096
3,(주)창원이엔지 홍제지점,02-379-0753,서울 서대문구 통일로 455 (홍제동),현대오일뱅크,NaN,"1,299","1,099",NaN,NaN,126.943338,37.589129
4,NaN,02-2665-5189,서울 강서구 개화동로 420 (개화동),현대오일뱅크,NaN,"1,264","1,064",NaN,NaN,126.802289,37.574341


## Folium

In [11]:
import pandas as pd
import numpy as np

# Python Data. Leaflet.js Maps.
import folium

In [12]:
df = pd.read_csv('./data/kakaomapapi.csv')
df.head(1)

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄,lon,lat
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN,126.846965,37.549279


In [13]:
mapping = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=13)

for i in df.index:
    folium.Marker(
        location=[df['lat'][i], df['lon'][i]],
        popup=df['이름'][i],
    ).add_to(mapping)

mapping